In [3]:
pip install stable-baselines3 gymnasium numpy torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
import gymnasium as gym
import numpy as np
import random
from gymnasium import spaces

class AdaptiveMixedCriticalityEnv(gym.Env):
    
    #Adaptive Mixed-Criticality scheduling environment with dynamic budget adjustment.
  
    def __init__(self):
        super(AdaptiveMixedCriticalityEnv, self).__init__()

        # Define tasks as [Execution Time, Deadline, Criticality Level, Budget]
        self.tasks = [
            [2, 5, 1, 2],  # Low-criticality task (LC)
            [3, 7, 1, 3],  # Low-criticality task (LC)
            [4, 6, 2, 4],  # High-criticality task (HC)
            [1, 4, 2, 1]    # High-criticality task (HC)
        ]

        self.state = np.zeros(len(self.tasks))  # Task completion state
        self.time = 0  # Simulation clock
        self.high_criticality_mode = False  # Mode flag

        # Observation space: time + task completion status + mode flag
        self.observation_space = spaces.Box(low=0, high=10, shape=(len(self.tasks) + 2,), dtype=np.float32)

        # Action space: Selecting a task to execute
        self.action_space = spaces.Discrete(len(self.tasks))

    def step(self, action):
       
        #Perform an action (execute a task), adjust budgets if needed, and return next state.
        
        exec_time, deadline, crit_level, budget = self.tasks[action]

        # Check if task exceeds its budget (overrun)
        if exec_time > budget:
            if crit_level == 1:  # Low-criticality task
                budget = max(1, budget - 1)  # Reduce budget dynamically
                reward = -5  # Penalty for exceeding budget
            else:  # High-criticality task
                self.high_criticality_mode = True  # Enter High-Criticality Mode
                reward = -10  # Heavy penalty for HC task overrun
        else:
            reward = 10 if crit_level == 2 else 5  # Reward for completing tasks normally

        # Mode Switching: Reduce low-criticality execution in High-Criticality Mode
        if self.high_criticality_mode and crit_level == 1:
            reward -= 3  # Penalty for LC task execution in High-Criticality Mode

        # Update system state
        self.state[action] = 1  # Task completed
        self.time += exec_time

        # Check if all tasks are completed or deadline is exceeded
        done = np.all(self.state == 1) or self.time > max(d for _, d, _, _ in self.tasks)

        # Observation includes current time, task completion status, and mode flag
        obs = np.append(self.state, [self.time, int(self.high_criticality_mode)])

        return obs, reward, done, False, {}

    def reset(self, seed=None, options=None):
        #Reset the environment to the initial state.
        
        self.state = np.zeros(len(self.tasks))
        self.time = 0
        self.high_criticality_mode = False
        return np.append(self.state, [self.time, int(self.high_criticality_mode)]), {}

    def render(self):
       
        #Print the current state for debugging.
        mode = "High-Criticality Mode" if self.high_criticality_mode else "Normal Mode"
        print(f"Time: {self.time}, Mode: {mode}, Task Status: {self.state}")


In [ ]:
from stable_baselines3 import DQN

# Create environment
env = AdaptiveMixedCriticalityEnv()

# Create and train DQN model
model = DQN("MlpPolicy", env, verbose=1, learning_rate=0.001, buffer_size=10000, batch_size=32, exploration_fraction=0.1)
model.learn(total_timesteps=10000)

# Save the trained model
model.save("dqn_amc_dynamic_scheduler")
print("Model trained and saved successfully.")

Streaming output truncated to the last 5000 lines.
|    learning_rate    | 0.001    |
|    loss             | 0.00592  |
|    n_updates        | 861      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.89     |
|    ep_rew_mean      | 48.4     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 748      |
|    fps              | 1053     |
|    time_elapsed     | 3        |
|    total_timesteps  | 3566     |
| train/              |          |
|    learning_rate    | 0.001    |
|    loss             | 0.00414  |
|    n_updates        | 866      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.89     |
|    ep_rew_mean      | 48.4     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 752      |
|    fps              | 1053     |
|   

In [5]:
# Load trained model
model = DQN.load("dqn_amc_dynamic_scheduler")

# Test environment with trained model
obs, _ = env.reset()
done = False

while not done:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, _, _ = env.step(action)
    env.render()  # Print task execution status and mode changes

Time: 4, Mode: Normal Mode, Task Status: [0. 0. 1. 0.]
Time: 5, Mode: Normal Mode, Task Status: [0. 0. 1. 1.]
Time: 6, Mode: Normal Mode, Task Status: [0. 0. 1. 1.]
Time: 7, Mode: Normal Mode, Task Status: [0. 0. 1. 1.]
Time: 8, Mode: Normal Mode, Task Status: [0. 0. 1. 1.]
